### MACTORENDS DOWNLOADER

In [92]:
import pandas as pd
import numpy as np
import requests
from datetime import date
import time
import streamlit
import matplotlib.pyplot as plt
import re
import json
from bs4 import BeautifulSoup as bs


In [93]:
url = 'https://www.macrotrends.net/stocks/charts/MMM/3m/financial-ratios'
response = requests.get(url)
response

<Response [200]>

In [109]:
soup = BeautifulSoup(response.text, 'html.parser')

In [116]:
p = re.compile(r' var originalData = (.*?);\r\n\r\n\r',re.DOTALL)
data = json.loads(p.findall(response.text)[0])

In [117]:
data

[{'field_name': "<a href='/stocks/charts/MMM/3m/current-ratio'>Current Ratio</a>",
  'popup_icon': '<div class=\'ajax-chart\' data-tipped-options="ajax: {data: { t: \'MMM\', s: \'current-ratio\', freq: \'A\', statement: \'financial-ratios\' }}"><i style=\'font-size:18px; color:#337ab7;\' class=\'fas fa-chart-bar\'></i></span></div>',
  '2020-12-31': '1.88500',
  '2019-12-31': '1.40650',
  '2018-12-31': '1.89250',
  '2017-12-31': '1.85730',
  '2016-12-31': '1.88550',
  '2015-12-31': '1.54340',
  '2014-12-31': '2.06290',
  '2013-12-31': '1.69820',
  '2012-12-31': '2.19840',
  '2011-12-31': '2.24960',
  '2010-12-31': '2.00610',
  '2009-12-31': '2.20440',
  '2008-12-31': '1.64380',
  '2007-12-31': '1.83480',
  '2006-12-31': '1.22160',
  '2005-12-31': '1.35830'},
 {'field_name': "<a href='/stocks/charts/MMM/3m/long-term-debt-capital'>Long-term Debt / Capital</a>",
  'popup_icon': '<div class=\'ajax-chart\' data-tipped-options="ajax: {data: { t: \'MMM\', s: \'long-term-debt-capital\', freq: 

In [97]:
headers = list(data[0].keys())
headers.remove('popup_icon')

In [98]:
headers

['field_name',
 '2020-12-31',
 '2019-12-31',
 '2018-12-31',
 '2017-12-31',
 '2016-12-31',
 '2015-12-31',
 '2014-12-31',
 '2013-12-31',
 '2012-12-31',
 '2011-12-31',
 '2010-12-31',
 '2009-12-31',
 '2008-12-31',
 '2007-12-31',
 '2006-12-31',
 '2005-12-31']

In [99]:
result = []

In [100]:
for row in data:
    soup = bs(row['field_name'])
    field_name = soup.select_one('a, span').text
    fields = list(row.values())[2:]
    fields.insert(0, field_name)
    result.append(fields)

pd.option_context('display.max_rows', None, 'display.max_columns', None)
df = pd.DataFrame(result, columns = headers)
print(df.head())

                 field_name 2020-12-31 2019-12-31 2018-12-31 2017-12-31  \
0             Current Ratio    1.88500    1.40650    1.89250    1.85730   
1  Long-term Debt / Capital    0.58180    0.63370    0.57660    0.51000   
2         Debt/Equity Ratio    1.45350    2.00600    1.48480    1.20020   
3              Gross Margin   48.40600   46.67660   49.08590   49.28450   
4          Operating Margin   22.25020   19.21210   21.99600   24.29790   

  2016-12-31 2015-12-31 2014-12-31 2013-12-31 2012-12-31 2011-12-31  \
0    1.88550    1.54340    2.06290    1.69820    2.19840    2.24960   
1    0.50800    0.43290    0.33780    0.19420    0.21410    0.22040   
2    1.12640    0.94150    0.51830    0.33480    0.33260    0.32570   
3   49.78910   49.18740   48.31400   47.82810   47.54880   47.00280   
4   23.33850   22.94380   22.42230   21.59310   21.67940   20.86390   

  2010-12-31 2009-12-31 2008-12-31 2007-12-31 2006-12-31 2005-12-31  
0    2.00610    2.20440    1.64380    1.83480    1.2

In [115]:
df

,field_name,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,2012-12-31,2011-12-31,2010-12-31,2009-12-31,2008-12-31,2007-12-31,2006-12-31,2005-12-31
0,Current Ratio,1.88500,1.40650,1.89250,1.85730,1.88550,1.54340,2.06290,1.69820,2.19840,2.24960,2.00610,2.20440,1.64380,1.83480,1.22160,1.35830
1,Long-term Debt / Capital,0.58180,0.63370,0.57660,0.51000,0.50800,0.43290,0.33780,0.19420,0.21410,0.22040,0.20710,0.27700,0.33390,0.25490,0.09510,0.11180
2,Debt/Equity Ratio,1.45350,2.00600,1.48480,1.20020,1.12640,0.94150,0.51830,0.33480,0.33260,0.32570,0.34040,0.42930,0.65200,0.41880,0.35680,0.22910
3,Gross Margin,48.40600,46.67660,49.08590,49.28450,49.78910,49.18740,48.31400,47.82810,47.54880,47.00280,48.12470,47.63220,47.05370,47.93970,48.90290,50.82910
4,Operating Margin,22.25020,19.21210,21.99600,24.29790,23.33850,22.94380,22.42230,21.59310,21.67940,20.86390,22.19640,20.81910,20.64980,25.31680,24.84840,22.93190
5,EBIT Margin,22.25020,19.21210,21.99600,24.29790,23.33850,22.94380,22.42230,21.59310,21.67940,20.86390,22.19640,20.81910,20.64980,25.31680,24.84840,22.93190
6,EBITDA Margin,28.18790,24.16920,26.53750,29.17520,28.23410,27.68380,26.84710,26.03410,25.98650,25.03800,26.39710,25.82280,25.21270,29.69910,29.55550,27.59010
7,Pre-Tax Profit Margin,20.85200,17.77450,21.36430,23.84310,23.42490,22.53750,22.07980,21.25620,21.23800,20.36740,21.58500,20.03200,20.21450,24.99800,24.53870,22.80910
8,Net Profit Margin,16.72880,14.22080,16.32540,15.34570,16.77240,15.96420,15.57460,15.09180,14.86090,14.46420,15.32140,13.80880,13.69270,16.74430,16.79970,14.69740
9,Asset Turnover,0.67980,0.71960,0.89770,0.83340,0.91500,0.92070,1.01960,0.92010,0.88270,0.93660,0.88410,0.84860,0.97970,0.99060,1.07650,1.03050
